# Data science challenge
Uitgevoerd door:
- Xin Wang
- Jens de Laat

## Opdracht:
House price predictor - Advanced

In [32]:
import pandas as pd  
import matplotlib as plt 
import seaborn as sns 
import numpy as np 
import sklearn as sk 
from sklearn.preprocessing import OrdinalEncoder

In [7]:
xins_path = "data/train.csv"
xins_test_path = "data/test.csv"

jens_train_path = "C:/Users/jensd/Desktop/Challange week/HousePricePredictor/data/train.csv"
jens_test_path = "C:/Users/jensd/Desktop/Challange week/HousePricePredictor/data/test.csv"

train_data = pd.read_csv(xins_path)
test_data = pd.read_csv(xins_test_path)

columns = train_data.columns.sort_values()

In [8]:
# Target variabel
target = ['SalePrice']
# Nutteloze kolommen
useless_columns = ['3SsnPorch', 'Alley', 'BldgType', 'BsmtCond',  'BsmtFinSF2', 'BsmtFinType2', 'BsmtQual', 'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'EnclosedPorch',
 'ExterCond','Functional', 'GarageCond', 'Heating', 'Id', "KitchenAbvGr", 'LandContour', 'LandSlope', 'LowQualFinSF',  'PavedDrive',
 'PoolArea','MiscFeature', 'MiscVal', "RoofMatl", "SaleCondition", "SaleType", "ScreenPorch", "Street", "TotalBsmtSF", "Utilities"]

# Numerieke kolommen
numeric_columns = ['1stFlrSF', '2ndFlrSF', 'BedroomAbvGr','BsmtFinSF1','BsmtFullBath', 'BsmtHalfBath','BsmtUnfSF','Fireplaces', 
       'FullBath', 'GarageArea', 'GarageCars', 'GrLivArea', 'HalfBath', 'LotArea', 'LotFrontage', 'MSSubClass', 
       'MasVnrArea', 'MoSold', 'OpenPorchSF', 'OverallCond', 'OverallQual', 'TotRmsAbvGrd', 'WoodDeckSF', 'YearBuilt',
       'YearRemodAdd', 'YrSold']

# Categorische kolommen
categorical_columns = [col for col in columns if (col not in numeric_columns) and col not in useless_columns and col not in target]

In [11]:
print(len(numeric_columns))
print(len(categorical_columns))

26
22


In [9]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [11]:
test_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [13]:
train_data.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [15]:
train_data.columns.sort_values()

Index(['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr',
       'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath',
       'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2',
       'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st',
       'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation',
       'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond',
       'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea',
       'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'Id', 'KitchenAbvGr',
       'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig',
       'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning',
       'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
       'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'Po

## Data exploratie 

### Numeriek VS Numeriek

In [52]:
locallist = numeric_columns + target + ["totalFlrSF"]
train_data["totalFlrSF"] = train_data["1stFlrSF"] + train_data["2ndFlrSF"] + train_data["BsmtFinSF1"] + train_data["BsmtUnfSF"]

In [53]:
numeric_correlation = train_data[locallist].corr()
numeric_correlation.style.background_gradient(cmap="coolwarm")

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtFinSF1,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,Fireplaces,FullBath,GarageArea,GarageCars,GrLivArea,HalfBath,LotArea,LotFrontage,MSSubClass,MasVnrArea,MoSold,OpenPorchSF,OverallCond,OverallQual,TotRmsAbvGrd,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SalePrice,totalFlrSF
1stFlrSF,1.000000,-0.202646,0.127401,0.445863,0.244671,0.001956,0.317987,0.410531,0.380637,0.489782,0.439317,0.566024,-0.119916,0.299475,0.457181,-0.251758,0.344501,0.031372,0.211671,-0.144203,0.476224,0.409516,0.235459,0.281986,0.240379,-0.013604,0.605852,0.773838
2ndFlrSF,-0.202646,1.000000,0.502901,-0.137079,-0.169494,-0.023855,0.004469,0.194561,0.421378,0.138347,0.183926,0.687501,0.609707,0.050986,0.080177,0.307886,0.174561,0.035164,0.208026,0.028942,0.295493,0.616423,0.092165,0.010308,0.140024,-0.028700,0.319334,0.358768
BedroomAbvGr,0.127401,0.502901,1.000000,-0.107355,-0.150673,0.046519,0.166643,0.107570,0.363252,0.065253,0.086106,0.521270,0.226651,0.119690,0.263170,-0.023438,0.102821,0.046544,0.093810,0.012980,0.101676,0.676620,0.046854,-0.070651,-0.040581,-0.036014,0.168213,0.353718
BsmtFinSF1,0.445863,-0.137079,-0.107355,1.000000,0.649212,0.067418,-0.495251,0.260011,0.058543,0.296970,0.224054,0.208171,0.004262,0.214103,0.233633,-0.069836,0.264736,-0.015727,0.111761,-0.046231,0.239666,0.044316,0.204306,0.249503,0.128451,0.014359,0.386420,0.421625
BsmtFullBath,0.244671,-0.169494,-0.150673,0.649212,1.000000,-0.147871,-0.422900,0.137928,-0.064512,0.179189,0.131881,0.034836,-0.030905,0.158155,0.100949,0.003491,0.085310,-0.025361,0.067341,-0.054942,0.111098,-0.053275,0.175315,0.187599,0.119470,0.067049,0.227122,0.156501
BsmtHalfBath,0.001956,-0.023855,0.046519,0.067418,-0.147871,1.000000,-0.095804,0.028976,-0.054536,-0.024536,-0.020891,-0.018918,-0.012340,0.048046,-0.007234,-0.002333,0.026673,0.032873,-0.025324,0.117821,-0.040150,-0.023836,0.040161,-0.038162,-0.012337,-0.046524,-0.016844,-0.025603
BsmtUnfSF,0.317987,0.004469,0.166643,-0.495251,-0.422900,-0.095804,1.000000,0.051575,0.288886,0.183303,0.214175,0.240257,-0.041118,-0.002618,0.132644,-0.140759,0.114442,0.034888,0.129005,-0.136841,0.308159,0.250647,-0.005316,0.149040,0.181133,-0.041258,0.214479,0.410868
Fireplaces,0.410531,0.194561,0.107570,0.260011,0.137928,0.028976,0.051575,1.000000,0.243671,0.269141,0.300789,0.461679,0.203649,0.271364,0.266639,-0.045569,0.249070,0.046357,0.169405,-0.023820,0.396765,0.326114,0.200019,0.147716,0.112581,-0.024096,0.466929,0.464089
FullBath,0.380637,0.421378,0.363252,0.058543,-0.064512,-0.054536,0.288886,0.243671,1.000000,0.405656,0.469672,0.630012,0.136381,0.126031,0.198769,0.131608,0.276833,0.055872,0.259977,-0.194149,0.550600,0.554784,0.187703,0.468271,0.439046,-0.019669,0.560664,0.585140
GarageArea,0.489782,0.138347,0.065253,0.296970,0.179189,-0.024536,0.183303,0.269141,0.405656,1.000000,0.882475,0.468997,0.163549,0.180403,0.344997,-0.098672,0.373066,0.027974,0.241435,-0.151521,0.562022,0.337822,0.224666,0.478954,0.371600,-0.027378,0.623431,0.561918


9 Kolommen hebben enige verband met de SalesPrice variabel. Dus 0.5 > Correlatie of Correlatie < -0.5
- GrLivArea
- OverallQual
- YearBuilt
- YearRemodAdd
- TotRmsAbvGrd
- FullBath
- GarageArea
- GarageCars
- 1stFlrSF (Misschien samenvoegen met 2ndFlrSF)

In [60]:
correlated_numeric_columns = ["GrLivArea",  "OverallQual", "YearBuilt",  "YearRemodAdd", "TotRmsAbvGrd", "FullBath", "GarageArea", "GarageCars", "1stFlrSF", "totalFlrSF"] + target

In [61]:
numeric_correlation = train_data[correlated_numeric_columns]

numeric_correlation.corr().style.background_gradient(cmap="coolwarm")

,GrLivArea,OverallQual,YearBuilt,YearRemodAdd,TotRmsAbvGrd,FullBath,GarageArea,GarageCars,1stFlrSF,totalFlrSF,SalePrice
GrLivArea,1.000000,0.593007,0.199010,0.287389,0.825489,0.630012,0.468997,0.467247,0.566024,0.867915,0.708624
OverallQual,0.593007,1.000000,0.572323,0.550684,0.427452,0.550600,0.562022,0.600671,0.476224,0.673284,0.790982
YearBuilt,0.199010,0.572323,1.000000,0.592855,0.095589,0.468271,0.478954,0.537850,0.281986,0.353373,0.522897
YearRemodAdd,0.287389,0.550684,0.592855,1.000000,0.191740,0.439046,0.371600,0.420622,0.240379,0.352782,0.507101
TotRmsAbvGrd,0.825489,0.427452,0.095589,0.191740,1.000000,0.554784,0.337822,0.362289,0.409516,0.673036,0.533723
FullBath,0.630012,0.550600,0.468271,0.439046,0.554784,1.000000,0.405656,0.469672,0.380637,0.585140,0.560664
GarageArea,0.468997,0.562022,0.478954,0.371600,0.337822,0.405656,1.000000,0.882475,0.489782,0.561918,0.623431
GarageCars,0.467247,0.600671,0.537850,0.420622,0.362289,0.469672,0.882475,1.000000,0.439317,0.538741,0.640409
1stFlrSF,0.566024,0.476224,0.281986,0.240379,0.409516,0.380637,0.489782,0.439317,1.000000,0.773838,0.605852
totalFlrSF,0.867915,0.673284,0.353373,0.352782,0.673036,0.585140,0.561918,0.538741,0.773838,1.000000,0.777018


### Categorische waarden

In [26]:
target
categorical_data = train_data[categorical_columns]

In [35]:
ordinalEncoder = OrdinalEncoder()
# BsmtExposure Ordinal
mappingBsmtExp = [["Ex", 5], ["Gd", 4], ["TA", 3], ["Fa", 2], ["Po", 1], ["NA", 0]]

# BsmtFinType1 Ordinal
mapBsmtFinT1 = [["GLQ", 6], ["ALQ", 5], ["BLQ", 4], ["Rec", 3], ["LwQ", 2], ["Unf", 1], ["NA", 0]]
# ExterQual Ordinal
mappingExtQual = [["Ex", 5], ["Gd", 4], ["TA", 3], ["Fa", 2], ["Po", 1]]
# Exterior1st One-Hot-Encoding
# Exterior2nd One-Hot-Encoding
# Fence Ordinal
mapFenceQual =  [["GdPrv", 4], ["MnPrv", 3], ["GdWo", 2], ["MnWw", 1], ["NA", 0]]
categorical_data["Fence"].fillna("NA", inplace=True)

C:\Users\Xin20\AppData\Local\Temp\ipykernel_18684\3083182474.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categorical_data["Fence"].fillna("NA", inplace=True)


In [36]:
ordinalEncoder.fit(mappingBsmtExp)

OrdinalEncoder()

In [38]:
ordinalEncoder.categories_

[array(['Ex', 'Fa', 'Gd', 'NA', 'Po', 'TA'], dtype=object),
 array([0, 1, 2, 3, 4, 5], dtype=object)]

In [59]:
ordinalEncoder.transform(categorical_data)

c:\Users\Xin20\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but OrdinalEncoder was fitted without feature names
  warnings.warn(


ValueError: X has 22 features, but OrdinalEncoder is expecting 2 features as input.